In [1]:
import openml, fairlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from keras.models import Sequential
from keras.layers import Dense
from fairlib.inprocessing import Fauci

INFO:fairlib:fairlib loaded
INFO:fairlib:Using Keras backend: TENSORFLOW


In [2]:
dataset = openml.datasets.get_dataset(179)
X, y, _, names = dataset.get_data(target=dataset.default_target_attribute)

INFO:openml.datasets.dataset:pickle write adult


In [3]:
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)

In [4]:
X_discretized = X_imputed.copy()
for col in X.columns:
    if X[col].dtype == 'category':
        le = LabelEncoder()
        X_discretized[:, X.columns.get_loc(col)] = le.fit_transform(X_discretized[:, X.columns.get_loc(col)])


In [5]:
adult = fairlib.DataFrame(X_discretized, columns=names)
adult['income'] = y
adult["income"] = adult["income"].apply(lambda x: x == ">50K").astype(int)
adult

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,income
0,2,6,77516.0,9,13,4,0,1,4,1,1,0,2,38,0
1,3,5,83311.0,9,13,2,3,0,4,1,0,0,0,38,0
2,2,3,215646.0,11,9,0,5,1,4,1,0,0,2,38,0
3,3,3,234721.0,1,7,2,5,0,2,1,0,0,2,38,0
4,1,3,338409.0,9,13,2,9,5,2,0,0,0,2,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,2,3,215419.0,9,13,0,9,1,4,0,0,0,2,38,0
48838,4,3,321403.0,11,9,6,9,2,2,1,0,0,2,38,0
48839,2,3,374983.0,9,13,2,9,0,4,1,0,0,3,38,0
48840,2,3,83891.0,9,13,0,0,3,1,1,2,0,2,38,0


In [6]:
adult.targets = "income"
adult.sensitive = 'sex' # fauci currently supports only one sensitive attribute

In [12]:
model = Sequential()
model.add(Dense(64, activation='relu'))  # Input layer and first hidden layer
model.add(Dense(32, activation='relu'))  # Second hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for classification

In [13]:
inprocessing = Fauci(model, loss='binary_crossentropy', regularizer='sp', optimizer='adam', metrics=['accuracy'])

In [14]:
inprocessing.fit(adult, converting_to_type=float, epochs=20, batch_size=52, validation_split=0.2)

this is X
[[2 6 77516.0 ... 0 2 38]
 [3 5 83311.0 ... 0 0 38]
 [2 3 215646.0 ... 0 2 38]
 ...
 [2 3 374983.0 ... 0 3 38]
 [2 3 83891.0 ... 0 2 38]
 [1 4 182148.0 ... 0 3 38]]
9
[1. 1. 1. ... 1. 1. 1.]
Epoch 1/20
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6374 - loss: 503.7008 - val_accuracy: 0.7636 - val_loss: 112.4635
Epoch 2/20
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6440 - loss: 236.1176 - val_accuracy: 0.2364 - val_loss: 82.8642
Epoch 3/20
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6290 - loss: 147.1021 - val_accuracy: 0.2364 - val_loss: 92.9905
Epoch 4/20
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6358 - loss: 125.8136 - val_accuracy: 0.7636 - val_loss: 243.3496
Epoch 5/20
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6378 - loss: 147.5412 - val_accuracy: 0.7636 - val_loss: 94.0597
Epoch 6/20
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6404 - loss: 126.4348 - val_accuracy: 0.7653 - val_loss: 37.6332
Epoch 7/20
